In [1]:
from clearml import Task, StorageManager, Dataset
import argparse, json, os, random, math

class bucket_ops:
    StorageManager.set_cache_file_limit(5, cache_context=None)

    def list(remote_path:str):
        return StorageManager.list(remote_path, return_full_path=False)

    def upload_folder(local_path:str, remote_path:str):
        StorageManager.upload_folder(local_path, remote_path, match_wildcard=None)
        print("Uploaded {}".format(local_path))

    def download_folder(local_path:str, remote_path:str):
        StorageManager.download_folder(remote_path, local_path, match_wildcard=None, overwrite=True)
        print("Downloaded {}".format(remote_path))
    
    def get_file(remote_path:str):        
        object = StorageManager.get_local_copy(remote_path)
        return object

    def upload_file(local_path:str, remote_path:str):
        StorageManager.upload_file(local_path, remote_path, wait_for_upload=True, retries=3)


In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from transformers import LongformerForMaskedLM, LongformerTokenizer

class mlm(torch.nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        #self.config =  
        self.softmax = nn.Softmax(dim=-1)
        self.tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
        #self.longformer = LongformerForMaskedLM.from_pretrained('allenai/longformer-base-4096', gradient_checkpointing=True, output_hidden_states=False)

        pretrained_lm_path = bucket_ops.get_file(
            remote_path="s3://experiment-logging/storage/ner-pretraining/NER-LM.c1a2da99836542849c6e8358498fed81/models/best_entity_lm.ckpt"
            )
        self.longformer = LongformerForMaskedLM.from_pretrained(pretrained_lm_path, gradient_checkpointing=True)
        

    def forward(self, **batch):
        # in lightning, forward defines the prediction/inference actions
        #input_ids, attention_mask, labels = batch
        outputs = self.longformer(**batch, return_dict=False)
        return outputs

args={
    "num_epochs":50,
    "train_batch_size":2,
    "eval_batch_size":4
}
args = argparse.Namespace(**args)
model = mlm(args)


ClearML Task: created new task id=c5064beda70e493a8d26efd151f6f785
ClearML results page: http://experiment.sytes.net:8080/projects/df20ee0d38af4ac787b6c77555163a03/experiments/c5064beda70e493a8d26efd151f6f785/output/log
ClearML new version available: upgrade to v1.1.2 is recommended!
2021-10-10 02:19:25,151 - clearml - WARNING - Switching to remote execution, output log page http://experiment.sytes.net:8080/projects/df20ee0d38af4ac787b6c77555163a03/experiments/c5064beda70e493a8d26efd151f6f785/output/log


In [ ]:
inputs = model.tokenizer("The capital of France is <mask>.", return_tensors="pt")

Error: Kernel is dead

In [ ]:
inputs["input_ids"]

In [ ]:
outputs = model(inputs)